In [ ]:
#%tensorflow_version 2.x
#TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import matplotlib.pyplot as plt
import os

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d paultimothymooney/breast-histopathology-images
!unzip -q breast-histopathology-images.zip -d IDC_images



Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images
License(s): CC0-1.0
100% 3.08G/3.10G [00:27<00:00, 94.5MB/s]
100% 3.10G/3.10G [00:27<00:00, 120MB/s] 


In [ ]:
import os
import glob
import cv2
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

# Step 1: Set path to extracted dataset
data_dir = '/content/IDC_images'  # This is where you unzipped the dataset
image_size = 50

X = []
y = []

# Step 2: Load all images and labels
for patient_folder in os.listdir(data_dir):
    for label in ['0', '1']:
        folder_path = os.path.join(data_dir, patient_folder, label)
        if not os.path.exists(folder_path):
            continue
        for img_file in glob.glob(os.path.join(folder_path, '*.png')):
            img = cv2.imread(img_file)
            img = cv2.resize(img, (image_size, image_size))
            X.append(img)
            y.append(int(label))  # Don't use one-hot here

X = np.array(X)
y = np.array(y)

# Step 3: Split into train/val/test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Step 4: Save images to directory format
base_dir = "/content/IDC_images_split"
for split in ['train', 'val', 'test']:
    for cls in ['0', '1']:
        os.makedirs(os.path.join(base_dir, split, cls), exist_ok=True)

def save_images(X_data, y_data, split_name):
    for i in range(len(X_data)):
        label = str(y_data[i])  # Already 0 or 1
        img = Image.fromarray(X_data[i].astype(np.uint8))  # CV2 loads as uint8
        path = os.path.join(base_dir, split_name, label, f"{label}_{i}.png")
        img.save(path)

save_images(X_train, y_train, 'train')
save_images(X_val, y_val, 'val')
save_images(X_test, y_test, 'test')


In [ ]:
print(f"Total images loaded: {len(X)}")
print(f"Shape of first image: {X[0].shape}")
print(f"Sample labels: {y[:10]}")


Total images loaded: 277524
Shape of first image: (50, 50, 3)
Sample labels: [0 0 0 0 0 0 0 0 0 0]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to directories
train_path = '/content/IDC_images_split/train'
val_path = '/content/IDC_images_split/val'
test_path = '/content/IDC_images_split/test'

# ImageDataGenerators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Flow from directories
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(50, 50),
    batch_size=64,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    val_path,
    target_size=(50, 50),
    batch_size=64,
    class_mode='binary',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    test_path,
    target_size=(50, 50),
    batch_size=64,
    class_mode='binary',
    shuffle=False
)


Found 194266 images belonging to 2 classes.
Found 41629 images belonging to 2 classes.
Found 41629 images belonging to 2 classes.


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# Define CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)),
    layers.MaxPooling2D((2, 2)),
    # layers.Dropout(0.1),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # layers.Dropout(0.1),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # layers.Dropout(0.1),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    # layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # Binary output
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Set early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# #Limit number of samples per class for speed (e.g., 10000/class for training)
# train_generator = train_datagen.flow_from_directory(
#     train_path,
#     target_size=(50, 50),
#     batch_size=64,
#     class_mode='binary',
#     shuffle=True,
#     subset='training'
# )

# validation_generator = val_test_datagen.flow_from_directory(
#     val_path,
#     target_size=(50, 50),
#     batch_size=64,
#     class_mode='binary',
#     shuffle=False
# )

# test_generator = val_test_datagen.flow_from_directory(
#     test_path,
#     target_size=(50, 50),
#     batch_size=64,
#     class_mode='binary',
#     shuffle=False
# )
# #limiting end

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

test_loss, test_acc = model.evaluate(test_generator)
print("✅ Test Accuracy:", test_acc)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 22, 22, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 11, 11, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 9, 9, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 355,649 (1.36 MB)

 Trainable params: 355,649 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3036/3036 ━━━━━━━━━━━━━━━━━━━━ 1105s 363ms/step - accuracy: 0.8089 - loss: 0.4311 - val_accuracy: 0.8478 - val_loss: 0.3559
Epoch 2/10
3036/3036 ━━━━━━━━━━━━━━━━━━━━ 1093s 360ms/step - accuracy: 0.8472 - loss: 0.3585 - val_accuracy: 0.8565 - val_loss: 0.3459
Epoch 3/10
3036/3036 ━━━━━━━━━━━━━━━━━━━━ 1039s 342ms/step - accuracy: 0.8557 - loss: 0.3395 - val_accuracy: 0.8577 - val_loss: 0.3358
Epoch 4/10
3036/3036 ━━━━━━━━━━━━━━━━━━━━ 1064s 350ms/step - accuracy: 0.8608 - loss: 0.3301 - val_accuracy: 0.8655 - val_loss: 0.3174
Epoch 5/10
3036/3036 ━━━━━━━━━━━━━━━━━━━━ 1031s 339ms/step - accuracy: 0.8627 - loss: 0.3249 - val_accuracy: 0.8649 - val_loss: 0.3199
Epoch 6/10
3036/3036 ━━━━━━━━━━━━━━━━━━━━ 1007s 332ms/step - accuracy: 0.8652 - loss: 0.3171 - val_accuracy: 0.8659 - val_loss: 0.3153
Epoch 7/10
3036/3036 ━━━━━━━━━━━━━━━━━━━━ 1044s 332ms/step - accuracy: 0.8665 - loss: 0.3150 - val_accuracy: 0.8674 - val_loss: 0.3158
Epoch 8/10
3036/3036 ━━━━━━━━━━━━━━━━━━━━ 991s 326ms/st